In [6]:
import xml.etree.ElementTree as ET
from xml.dom import minidom

In [7]:
def prettify(elem):
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [9]:
ns = {"bsim": "http://bsim.hpi.uni-potsdam.de/scylla/simModel"}

ET.register_namespace('bsim', ns["bsim"])
root = ET.Element(ET.QName(ns["bsim"], "globalConfiguration"))

resourceData =  ET.Element(ET.QName(ns["bsim"], "resourceData"))
root.append(resourceData)
# For each role
for role in ["Sachbearbeiter", "Manager"]:
    dynamicResource =  ET.Element(ET.QName(ns["bsim"], "dynamicResource"))
    dynamicResource.set("id", role)
    dynamicResource.set("defaultQuantity", "5")
    resourceData.append(dynamicResource)


In [10]:
print(prettify(root))

<?xml version="1.0" ?>
<bsim:globalConfiguration xmlns:bsim="http://bsim.hpi.uni-potsdam.de/scylla/simModel">
  <bsim:resourceData>
    <bsim:dynamicResource defaultQuantity="5" id="Sachbearbeiter"/>
    <bsim:dynamicResource defaultQuantity="5" id="Manager"/>
  </bsim:resourceData>
</bsim:globalConfiguration>

